# <div align='center'> K12学习框架架构 </div>

## 1. 架构图

```
                                          +-----------------------------------------------------+
                                          |                                                     |
                                          |   +---------------+             +---------------+   |
           +----------------------------->|   |  container-1  |     ...     |  container-2  |   |
           |                              |   +---------------+             +---------------+   |
           |                              |             ^                        ^      |       |
           | stats                        |              \  docker       docker /       |       |
           | control                      |               \                    /        |       |
           |                              +-----------------------------------------------------+
           |                                                \                /          |
  service  |                   service                       \              /           | RPC
  +-----------------+ get      +----------------+            +---------------+          |
  |RPC              |--------+ |RPC             |            |RPC            |<---------+
  |   k12platform   |        | |      k12cv     |      +-----|    k12nlp     |
  |                 |<----+  | |                |      |     |               |----------+
  +-----------------+     |  | +----------------+      |     +---------------+          |
          ^     |      ip |  |           |             |         |      ^               |
          |     |     port|  |           |regist       |dev      |      |               |
          |     |         |  |           |             |         |      |               |http
          |     |         |  v           v             v         |      |               |
          |     |       +---------------------------------+      |      |               |
          |     |       |                       datacenter|      |      |               |
          |     +------>|              CONSUL             |<-----+      |               |
      RPC |     regist  |service health                   | regist      | RPC           |
          |             +---------------------------------+             |               |
          |                   | ^        ^                              |               |
          |               ip  | |        |                              |               |
          |              port | |get     | regist                       |               |
          |                   v |        |                              |               |
        +-----------------------------------------------------------------+             |
        |  HTTP                                                           |             |
        |                 k12ai (flask service api)                       | <-----------+
        |                                                                 |
        +-----------------------------------------------------------------+
                                      |  ^
                                      |  |
                                redis |  | http
                                      |  |
                                      v  |
                                   *********
                                ***         ***
                              **               **
                              *    Data / OP    *
                              **               **
                                ***         ***
                                   *********
```                                   

## 2. 描述

- 框架中的Consul服务, 是内部服务之间通信的核心服务, 所有其他服务如(k12ai, k12nlp)启动时都会将自己注册到Consul服务中, Consul服务可以对已注册的服务进行健康(连通性)检测, 并提供[UI页面](http://gamma:8500/ui/gamma/services)展示已经注册服务的信息(服务的名字, 健康检测周期, Tag等).

> **注意: Consul服务目前只作为本地服务使用, 虽然可以部署到外部, 暂时没这个需求和应用要部署到外部**

- k12ai api服务是唯一一个与外界交互的服务, 提供restapi接口, 接收外部http请求如(train,constrol), 根据请求的任务不同, 查询对应的服务, 大体流程: 1. k12ai通过服务名字从Consul获取到请求中的服务(返回服务的ip和port), 如果服务没启动或不存在,提示错误. 2. 获取到服务(如, k12nlp)的ip和prot后, 通过RCP方式(agent代理方式)调用真正服务器的接口.


- k12nlp/k12cv 服务负责实现k12ai的代理所需操作的接口, 因为k12ai学习框架本身是个学习框架的集合, 每个框架的调用形式,风格各不相同, 这也是k12nlp, k12cv,...存在的意义, 简化k12ai服务(专心接收外部api调用), 复杂的框架应用过程由各自的框架服务, 如k12nlp去完成. 大体流程: 1. 对k12ai传来的参数进行处理, 准备(自己)框架运行的必要环境  2. 环境准备完成, 通过docker运行任务相关的container, container里运行的进程实际上是真正处理用户请求的主体.


- k12platform 服务提供管理平台的接口, 比如: 获取主机的CPU/GPU/MEM占用率, 每个container的CPU/MEM使用情况, 并可以进行查杀.


- container 提供任务进程执行的环境, 后台执行, 进程运行过程中的中间输出, 如: mertrics, error等通过RCP的方式汇报给启动它的框架服务,如: k12nlp, 该服务做一些处理之后, 通过http请求方式交给**k12ai**(唯一一个与外界交互的服务), 如果服务处在开发者模式(DEBUG)状态, 这些数据同时也会上传到Consul的数据中心上,便于开发者查看调试.